In [1]:
import pandas as pd
from pandas_datareader import data as wb
from yahoo_fin.stock_info import *
import requests
import plotly.graph_objects as go
import yfinance as yf
import numpy as np
import numpy_financial as npf
import lxml
from lxml import html
from bs4 import BeautifulSoup as bs
BS = []
IS = []
cfs = []
years = []
summary = []

In [2]:
def get_data(stock):
    global BS
    global IS
    global cfs
    global years
    global summary
    IS=get_income_statement(stock)
    BS=get_balance_sheet(stock)
    cfs=get_cash_flow(stock)
    years=BS.columns
    summary=get_quote_table(stock)


In [31]:
def DCF(stock):
    get_data(stock)
    risk_free=0.0169
    risk_premium=0.056
    g=0.02
    beta=summary['Beta (5Y Monthly)']
    cost_of_equity=risk_free+risk_premium*beta
    #cost_of_debt=float(input('What is the bond yield of ' + str(stock) + ' ? '))
   
    
    revenue_g=(IS[years[0]]['totalRevenue']-IS[years[1]]['totalRevenue'])/IS[years[1]]['totalRevenue']
    #kako revenue growth go zeame fiksno posledniot godisen zabelezan growth na 
    #revenue i toj ostanuva konstanten preku site godini
    income_statement = pd.DataFrame.from_dict(IS[years[0]])
    income_statement.columns=['current_year']
    
    if income_statement['current_year']['interestExpense']==None:
        income_statement['current_year']['interestExpense']=int(input('Enter the Interest Expense of the latest year of ' + stock + ' '))
        income_statement['current_year']['interestExpense']=income_statement['current_year']['interestExpense']*-1
    
    #praenje na data frame i koristenje samo na poslednata godina
    #pretstavuvanje na site elementi vo income statementot kako procent od revenue
    income_statement['as_%_of_revenue']=income_statement/income_statement['current_year']['totalRevenue']
    #posle ednakvoto prviot income_statement sto e, poradi to shto nema nikakvi 
    #tags za koloni i redovi raboti za celiot income_statement sto znaci deka 
    #operacijava ke se izvrsi za site elementi vo income_statementot
    
    #forecasting the next 5 years in the income statement
    income_statement['next_year'] =  (income_statement['current_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
    income_statement['next_2_year'] =  (income_statement['next_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
    income_statement['next_3_year'] =  (income_statement['next_2_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
    income_statement['next_4_year'] =  (income_statement['next_3_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
    income_statement['next_5_year'] =  (income_statement['next_4_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue']
    #raboti na princip kade sto sekoja godina se polni kako vo for loop za site 
    #elementi so to sto totalRevenue se mnozi so revenue growth i to se mnozi so
    #procentot od totalRevenue sekoj element sto go ima za sekoja od godinite

    balance_sheet = pd.DataFrame.from_dict(BS[years[0]])
    balance_sheet.columns=['current_year']
    balance_sheet['as_%_of_revenue']=balance_sheet/income_statement['current_year']['totalRevenue']
    
    #forecasting the next 5 years in the Balance Sheet.
    balance_sheet['next_year'] =  income_statement['next_year']['totalRevenue'] * balance_sheet['as_%_of_revenue']
    balance_sheet['next_2_year'] =  income_statement['next_2_year'] ['totalRevenue'] * balance_sheet['as_%_of_revenue']
    balance_sheet['next_3_year'] =  income_statement['next_3_year']['totalRevenue'] * balance_sheet['as_%_of_revenue'] 
    balance_sheet['next_4_year'] =  income_statement['next_4_year']['totalRevenue']  * balance_sheet['as_%_of_revenue'] 
    balance_sheet['next_5_year'] =  income_statement['next_5_year']['totalRevenue'] * balance_sheet['as_%_of_revenue']
    #principot na predvidvenje e ist ko kaj income statement za sekoj od 
    #elementi na balance sheetot im naprajvme % of revenue i spored to pomnozeno
    #so total revenue za taa godina ja dobivame vrednosta na toj element
    
    cash_flow = pd.DataFrame.from_dict(cfs[years[0]])
    cash_flow.columns=['current_year']
    
    depreciation_per = cash_flow['current_year']['depreciation']/income_statement['current_year']['totalRevenue']
    depreciation_fut = {}
    depreciation_fut['next_year'] = income_statement['next_year']['totalRevenue']*depreciation_per
    depreciation_fut['next_2_year'] = income_statement['next_2_year']['totalRevenue']*depreciation_per
    depreciation_fut['next_3_year'] = income_statement['next_3_year']['totalRevenue']*depreciation_per
    depreciation_fut['next_4_year'] = income_statement['next_4_year']['totalRevenue']*depreciation_per
    depreciation_fut['next_5_year'] = income_statement['next_5_year']['totalRevenue']*depreciation_per
    
    #ova se e spored TESLA DCF sho go praev vo excel za forecasting the free cash flow
    nopat={}
    gross_cash_flow={}
    inc_receivables={}
    inc_inventory={}
    inc_payables={}
    inv_working_capital={}
    capex={}
    other_assets_liabilities={}
    free_cash_flow_forecast={}
    
    #current year
    nopat['current_year']=income_statement['current_year']['ebit']-income_statement['current_year']['incomeTaxExpense']
    gross_cash_flow['current_year']=nopat['current_year']+cash_flow['current_year']['depreciation']
    
    inc_receivables['current_year']=BS[years[0]]['netReceivables'] - BS[years[1]]['netReceivables']
    try:
        inc_inventory['current_year']=BS[years[0]]['inventory'] - BS[years[1]]['inventory']
    except:
        pass
    inc_payables['current_year']=BS[years[0]]['accountsPayable'] - BS[years[1]]['accountsPayable']  
    
    try:
        inv_working_capital['current_year']=(inc_receivables['current_year'] *-1)+(inc_inventory['current_year']*-1)+inc_payables['current_year']
    except:
        inv_working_capital['current_year']=(inc_receivables['current_year'] *-1)+inc_payables['current_year']
        pass
    capex['current_year']=BS[years[0]]['propertyPlantEquipment']-BS[years[1]]['propertyPlantEquipment']+cash_flow['current_year']['depreciation']
    
    free_cash_flow_low = gross_cash_flow['current_year'] + inv_working_capital['current_year'] - capex['current_year']
    real_current_free_cash=cash_flow['current_year']['totalCashFromOperatingActivities']+cash_flow['current_year']['capitalExpenditures']
    other_assets_liabilities['current_year'] = real_current_free_cash - free_cash_flow_low
    
    other_per_revenue = other_assets_liabilities['current_year'] / income_statement['current_year']['totalRevenue']
    
    #formuli za presmetvenje na free cash flow za sekoja godina i datata za sekoja
    #godina se skladira vo dictionaries. Za current year ne ni treba free cash
    #flow, nego go koristime samo za da najme percentage na other assets i 
    #liabilities od revenue za spored toj procent da gi procenime za idnite godini
    
    #next year
    nopat['next_year']=income_statement['next_year']['ebit']-income_statement['next_year']['incomeTaxExpense']
    gross_cash_flow['next_year']=nopat['next_year']+depreciation_fut['next_year']
    
    inc_receivables['next_year']=balance_sheet['next_year']['netReceivables'] - balance_sheet['current_year']['netReceivables']
    try:
        inc_inventory['next_year']=balance_sheet['next_year']['inventory'] - balance_sheet['current_year']['inventory']
    except:
        pass
    inc_payables['next_year']=balance_sheet['next_year']['accountsPayable'] - balance_sheet['current_year']['accountsPayable']
    
    try:
        inv_working_capital['next_year']=(inc_receivables['next_year'] *-1)+(inc_inventory['next_year']*-1)+inc_payables['next_year']
    except:
        inv_working_capital['next_year']=(inc_receivables['next_year'] *-1)+inc_payables['next_year']
        pass
    capex['next_year']=balance_sheet['next_year']['propertyPlantEquipment']-balance_sheet['current_year']['propertyPlantEquipment']+depreciation_fut['next_year']
    other_assets_liabilities['next_year'] = income_statement['next_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_year']= gross_cash_flow['next_year'] + inv_working_capital['next_year'] - capex['next_year'] + other_assets_liabilities['next_year']
    
    #next 2 year
    nopat['next_2_year']=income_statement['next_2_year']['ebit']-income_statement['next_2_year']['incomeTaxExpense']
    gross_cash_flow['next_2_year']=nopat['next_2_year']+depreciation_fut['next_2_year']
    
    inc_receivables['next_2_year']=balance_sheet['next_2_year']['netReceivables'] - balance_sheet['next_year']['netReceivables']
    try:
        inc_inventory['next_2_year']=balance_sheet['next_2_year']['inventory'] - balance_sheet['next_year']['inventory']
    except:
        pass
    inc_payables['next_2_year']=balance_sheet['next_2_year']['accountsPayable'] - balance_sheet['next_year']['accountsPayable']
    
    try:
        inv_working_capital['next_2_year']=(inc_receivables['next_2_year'] *-1)+(inc_inventory['next_2_year']*-1)+inc_payables['next_2_year']
    except:
        inv_working_capital['next_2_year']=(inc_receivables['next_2_year'] *-1)+inc_payables['next_2_year']
        pass
    capex['next_2_year']=balance_sheet['next_2_year']['propertyPlantEquipment']-balance_sheet['next_year']['propertyPlantEquipment']+depreciation_fut['next_2_year']
    other_assets_liabilities['next_2_year'] = income_statement['next_2_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_2_year']= gross_cash_flow['next_2_year'] + inv_working_capital['next_2_year'] - capex['next_2_year'] + other_assets_liabilities['next_2_year']
    
    #next 3 year
    nopat['next_3_year']=income_statement['next_3_year']['ebit']-income_statement['next_3_year']['incomeTaxExpense']
    gross_cash_flow['next_3_year']=nopat['next_3_year']+depreciation_fut['next_3_year']
    
    inc_receivables['next_3_year']=balance_sheet['next_3_year']['netReceivables'] - balance_sheet['next_2_year']['netReceivables']
    try:
        inc_inventory['next_3_year']=balance_sheet['next_3_year']['inventory'] - balance_sheet['next_2_year']['inventory']
    except:
        pass
    inc_payables['next_3_year']=balance_sheet['next_3_year']['accountsPayable'] - balance_sheet['next_2_year']['accountsPayable']
    
    try:
        inv_working_capital['next_3_year']=(inc_receivables['next_3_year'] *-1)+(inc_inventory['next_3_year']*-1)+inc_payables['next_3_year']
    except:
        inv_working_capital['next_3_year']=(inc_receivables['next_3_year'] *-1)+inc_payables['next_3_year']
        pass
    capex['next_3_year']=balance_sheet['next_3_year']['propertyPlantEquipment']-balance_sheet['next_2_year']['propertyPlantEquipment']+depreciation_fut['next_3_year']
    other_assets_liabilities['next_3_year'] = income_statement['next_3_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_3_year']= gross_cash_flow['next_3_year'] + inv_working_capital['next_3_year'] - capex['next_3_year'] + other_assets_liabilities['next_3_year']
    
    #next 4 year
    nopat['next_4_year']=income_statement['next_4_year']['ebit']-income_statement['next_4_year']['incomeTaxExpense']
    gross_cash_flow['next_4_year']=nopat['next_4_year']+depreciation_fut['next_4_year']
    
    inc_receivables['next_4_year']=balance_sheet['next_4_year']['netReceivables'] - balance_sheet['next_3_year']['netReceivables']
    try:
        inc_inventory['next_4_year']=balance_sheet['next_4_year']['inventory'] - balance_sheet['next_3_year']['inventory']
    except:
        pass
    inc_payables['next_4_year']=balance_sheet['next_4_year']['accountsPayable'] - balance_sheet['next_3_year']['accountsPayable']
    
    try:
        inv_working_capital['next_4_year']=(inc_receivables['next_4_year'] *-1)+(inc_inventory['next_4_year']*-1)+inc_payables['next_4_year']
    except:
        inv_working_capital['next_4_year']=(inc_receivables['next_4_year'] *-1)+inc_payables['next_4_year']
        pass
    capex['next_4_year']=balance_sheet['next_4_year']['propertyPlantEquipment']-balance_sheet['next_4_year']['propertyPlantEquipment']+depreciation_fut['next_4_year']
    other_assets_liabilities['next_4_year'] = income_statement['next_4_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_4_year']= gross_cash_flow['next_4_year'] + inv_working_capital['next_4_year'] - capex['next_4_year'] + other_assets_liabilities['next_4_year']
    
    #next 5 year
    nopat['next_5_year']=income_statement['next_5_year']['ebit']-income_statement['next_5_year']['incomeTaxExpense']
    gross_cash_flow['next_5_year']=nopat['next_5_year']+depreciation_fut['next_5_year']
    
    inc_receivables['next_5_year']=balance_sheet['next_5_year']['netReceivables'] - balance_sheet['next_4_year']['netReceivables']
    try:
        inc_inventory['next_5_year']=balance_sheet['next_5_year']['inventory'] - balance_sheet['next_4_year']['inventory']
    except:
        pass
    inc_payables['next_5_year']=balance_sheet['next_5_year']['accountsPayable'] - balance_sheet['next_4_year']['accountsPayable']
    
    try:
        inv_working_capital['next_5_year']=(inc_receivables['next_5_year'] *-1)+(inc_inventory['next_5_year']*-1)+inc_payables['next_5_year']
    except:
        inv_working_capital['next_5_year']=(inc_receivables['next_5_year'] *-1)+inc_payables['next_5_year']
        pass
    capex['next_5_year']=balance_sheet['next_5_year']['propertyPlantEquipment']-balance_sheet['next_5_year']['propertyPlantEquipment']+depreciation_fut['next_5_year']
    other_assets_liabilities['next_5_year'] = income_statement['next_5_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_5_year']= gross_cash_flow['next_5_year'] + inv_working_capital['next_5_year'] - capex['next_5_year'] + other_assets_liabilities['next_5_year']
    
    #calculating the WACC TESLA DCF way
    
    #equity=wb.get_quote_yahoo('AAPL')['marketCap']
    #debt1=balance_sheet['next_year']['longTermDebt']/(balance_sheet['next_year']['longTermDebt']+equity)
    #equity1=equity/(balance_sheet['next_year']['longTermDebt']+equity)
    #wacc = debt1 * cost_of_debt * (1-0.21) + equity1 * cost_of_equity
    
    #calculating the WACC guru way
    
    equity=wb.get_quote_yahoo('AAPL')['marketCap']
    #getting the equity as a market cap
    balance_sheet_url = (f'https://finance.yahoo.com/quote/{stock}/balance-sheet?p={stock}')
    balance_sheet_html = requests.get(balance_sheet_url)
    balance_sheet_soup = bs(balance_sheet_html.text, 'html.parser')
    balance_sheet_table = balance_sheet_soup.find('div', class_='D(tbrg)')

    total_debt_lst = []
    total_debt_row = balance_sheet_table.find('div', attrs={'title':'Total Debt'}).parent.parent
    for value in total_debt_row.find_all('div'):
        value = value.text
        value = value.replace(',','')
        total_debt_lst.append(value)
    total_debt_int = int(total_debt_lst[3])
    total_debt_int=total_debt_int*1000
    #ova e kodot za zemanje na total debt od yahoo finance preku beautiful soup
    #princip ja zeame balance sheet stranata html, barame klasata na tabelata
    #so podatocite sto ni trebaat i od niv vlecime samo total debt i na kraj 
    #posledniot total debt go konvertirame vo int
    #total_debt_int go mnozime so 1000 radi to sho na yahoo finance podatocite
    #se vo thousands
    
    
        
    equity1=equity/(total_debt_int+equity)
    debt1=total_debt_int/(total_debt_int+equity)
    cost_of_debt=(income_statement['current_year']['interestExpense']*-1)/total_debt_int        
    wacc=debt1*cost_of_debt*(1-0.21)+equity1*cost_of_equity
    #brojkive vo zagradava kaj wacc se za tax rate go zaokruzvime na 21%
    
    wacc_stock=wacc[0]
    values=free_cash_flow_forecast.values()
    fcf_list=list(values)
    npv=npf.npv(wacc_stock,fcf_list)
    #calculating the net present value of the sum of all the free cash flows forecasted
    
    Terminal_value=(free_cash_flow_forecast['next_5_year']*(1+g))/(wacc_stock-g)
    Terminal_value_Discounted = Terminal_value/(1+wacc_stock)**5
    Terminal_value_Discounted
    #calculating the terminal value, i.e. the value of the company after 5 years
    
    enterprise_value=npv+Terminal_value_Discounted
    cash=balance_sheet['current_year']['cash']
    equity_value=enterprise_value+cash-total_debt_int
    
    shares=wb.get_quote_yahoo(stock)['sharesOutstanding']
    forecasted_price=equity_value/shares
    
    forecasted_price_pure=forecasted_price[0]
    equity_value_mln=equity_value/1000000
    real_price=wb.get_quote_yahoo(stock)['price']
    real_price_pure=real_price[0]
    
    print(stock + ' Current price per stock is ' + str(real_price_pure) )
    print(stock + ' forecasted price per stock is ' + str(forecasted_price_pure) )
    print('Equity value of '+ stock + " is " + str(equity_value_mln) + " mln" )
    print('the forecast is based on the following assumptions: '+ 'revenue growth: ' + str(revenue_g) + ' Cost of Capital: ' + str(wacc_stock) )
    print('perpetuity growth: ' + str(g))
    
    

In [32]:
while True:
    command=input('Write a stock ticker ')
    #IS AAPL - example of text inserted
    try:
        stock=command.split(' ')[1]
    #vo stock variable go delime stringot, t.e. inputot sho go vnesvime i 
    #mestoto na podelba e praznoto mesto i go zeame vtoriot del od splitot
    except:
        pass
                
    if command =='DCF ' + stock:
        DCF(stock)
               
    elif command=='quit':
        break
        
    else:
        print('Statement is not valid')

Write a stock ticker DCF AAPL
AAPL Current price per stock is 134.84
AAPL forecasted price per stock is 84.37606626461354
Equity value of AAPL is 1416513.846157061 mln
the forecast is based on the following assumptions: revenue growth: 0.055120803769784836 Cost of Capital: 0.08214269150673432
perpetuity growth: 0.02
Write a stock ticker quit


In [4]:
i=get_income_statement('DB', yearly=True)
i.iat[2,2]
#za konkretno vaganje na element od pozicija

1331000000

In [ ]:
i.keys()
#showing column indexes

In [5]:
years=i.columns
#storing the column names in variavle years for accessing of data

In [6]:
i[years[0]]
#printing the latest year of the company

Breakdown
researchDevelopment                         None
effectOfAccountingCharges                   None
incomeBeforeTax                       1021000000
minorityInterest                      1587000000
netIncome                              495000000
sellingGeneralAdministrative         19786000000
grossProfit                          22236000000
ebit                                           0
operatingIncome                       2450000000
otherOperatingExpenses                       NaN
interestExpense                             None
extraordinaryItems                          None
nonRecurring                                None
otherItems                                  None
incomeTaxExpense                       397000000
totalRevenue                         22236000000
totalOperatingExpenses               19786000000
costOfRevenue                                  0
totalOtherIncomeExpenseNet           -1429000000
discontinuedOperations                      None
netIncomeF

In [7]:
i[years[0]]['ebit']
#accessing a particular element from the first column and the name index
#of the row

0

In [45]:
test1=get_quote_table('AAPL')
print(test1['Beta (5Y Monthly)'])
#ova e za zemanje data od yahoo finance summary page, konkretno za extract na
#vrednosta na beta kolku e

1.22


Note to self, opcii za vnesuvanje na tax rate ko input
--------------------------

In [114]:
test2=wb.get_quote_yahoo('AAPL')['marketCap']
test3=get_balance_sheet('MSFT')
print(BS)

endDate                    2020-09-26    2019-09-28    2018-09-29  \
Breakdown                                                           
totalLiab                258549000000  248028000000  258578000000   
totalStockholderEquity    65339000000   90488000000  107147000000   
otherCurrentLiab          47867000000   43242000000   39293000000   
totalAssets              323888000000  338516000000  365725000000   
commonStock               50779000000   45174000000   40201000000   
otherCurrentAssets        11264000000   12352000000   12087000000   
retainedEarnings          14966000000   45898000000   70400000000   
otherLiab                 46108000000   50503000000   48914000000   
treasuryStock              -406000000    -584000000   -3454000000   
otherAssets               33952000000   32978000000   22283000000   
cash                      38016000000   48844000000   25913000000   
totalCurrentLiabilities  105392000000  105718000000  115929000000   
shortLongTermDebt          8773000

In [17]:
bala=requests.get('https://finance.yahoo.com/quote/AAPL/balance-sheet?p=AAPL')
tree = html.fromstring(bala.content)
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
print(bala)
print(tree)
print(table_rows)

<Response [200]>
<Element html at 0x2114eaaf770>
[<Element div at 0x2114eaaf5e0>, <Element div at 0x2114eaaf6d0>, <Element div at 0x2114eaaf4f0>, <Element div at 0x2114eaafe00>, <Element div at 0x2114eaafbd0>, <Element div at 0x2114eaaf950>, <Element div at 0x2114eaafdb0>, <Element div at 0x2114eaafe50>, <Element div at 0x2114eaaf360>, <Element div at 0x2114eaafd10>, <Element div at 0x2114eaaf630>, <Element div at 0x2114eaafcc0>, <Element div at 0x2114eaafc70>, <Element div at 0x2114eaafc20>]


In [11]:
url = ('https://finance.yahoo.com/quote/AAPL/balance-sheet?p=AAPL')
html_content = requests.get(url).text
# Make a GET request to fetch the raw HTML content

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) 
# print the parsed data of html
#comentiraniov kod gore e za cela html data da ja prikazi

#for link in soup.find_all("a"):
 #   print("Inner Text: {}".format(link.text))
  #  print("Title: {}".format(link.get("title")))
   # print("href: {}".format(link.get("href")))
#za zemanje na site linkovi vo stranata
    


In [19]:
#balance_sheet_url = '<https://finance.yahoo.com/quote/>' + stock + '/balance-sheet?p=' + stock
#ova e primer za kako spored variabla da go zeam

balance_sheet_url = 'https://finance.yahoo.com/quote/AAPL/balance-sheet?p=AAPL'
balance_sheet_html = requests.get(balance_sheet_url)
balance_sheet_soup = bs(balance_sheet_html.text, 'html.parser')

balance_sheet_table = balance_sheet_soup.find('div', class_='D(tbrg)')

total_debt_lst = []

total_debt_row = balance_sheet_table.find('div', attrs={'title':'Total Debt'}).parent.parent
for value in total_debt_row.find_all('div'):
    value = value.text
    value = value.replace(',','')
    total_debt_lst.append(value)
total_debt_int = int(total_debt_lst[3])

print(total_debt_lst)
print(total_debt_int)
#ova e kodot za zemanje na total debt od yahoo finance preku beautiful soup
#princip ja zeame balance sheet stranata html, barame klasata na tabelata
#so podatocite sto ni trebaat i od niv vlecime samo total debt i na kraj 
#posledniot total debt go konvertirame vo int

['Total Debt', 'Total Debt', '', '112436000', '108047000', '114483000', '115680000']
112436000


In [90]:
test4=wb.get_quote_yahoo('AAPL')['price']
print(test4)

AAPL    134.16
Name: price, dtype: float64
